In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
# import seaborn as sb
from tqdm import tqdm
import xarray as xr

from gogoesgone import processing as pr
from gogoesgone import zarr_access as za

year = 2020
month = 1
dayofyear = 1
channel = 13
product = "ABI-L2-CMIPF" 
satellite = "goes16"  
output_dir = '/Users/acmsavazzi/Documents/WORK/PhD_Year3/DATA/gogoesgone/output/'
# time = "20200101 12:00:00"

year = 2020
month = 1
channel = 13
product = "ABI-L2-CMIPF"  #'ABI-L1b-RadF'
satellite = "goes16"  

output_dir = '/Users/acmsavazzi/Documents/WORK/PhD_Year3/DATA/gogoesgone/output/'
# time = "20200101 12:00:00"
all_subsets = []  # List to store each subset
for dayofyear in tqdm(range(1, 15), desc="Processing days"):
    for hour in tqdm(range(1, 24), desc="Processing hours", leave=False):
        time = str(year)+str(month).zfill(2)+str(dayofyear).zfill(2)+' '+str(hour).zfill(2)+':00:00'
        flist = za.nearest_time_url(time)
        m = za.get_mapper_from_mzz(flist)
        img = pr.Image(m)
        extent = (-59, -56, 12, 14)
        subset = img.subset_region_from_latlon_extents(extent, unit="degree")
        all_subsets.append(subset)

# Concatenate along the 't' dimension and save to a single .nc file
concatenated = xr.concat(all_subsets, dim='t')
concatenated.rename({'t':'time'})
concatenated['CMI'].to_netcdf(output_dir + f'/goes_{year}{str(month).zfill(2)}.nc')


Processing days:   0%|          | 0/1 [00:00<?, ?it/s]/Users/acmsavazzi/opt/anaconda3/envs/gogoesgone/lib/python3.12/site-packages/kerchunk/combine.py:370: UserWarning: Concatenated coordinate 't' contains less than expectednumber of values across the datasets: [6.31523107e+08]
  warnings.warn(
/Users/acmsavazzi/opt/anaconda3/envs/gogoesgone/lib/python3.12/site-packages/xarray/backends/api.py:671: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  backend_ds = backend.open_dataset(
/Users/acmsavazzi/opt/anaconda3/envs/g

In [30]:
goes_ds = xr.open_dataset('/Users/acmsavazzi/Documents/WORK/PhD_Year3/DATA/gogoesgone/output/goes_202001_1.nc')

In [32]:
goes_ds.rename({'t':'time'})

<xarray.Dataset> Size: 547kB
Dimensions:  (x: 159, y: 107, time: 2)
Coordinates:
  * x        (x) float64 1kB 0.04707 0.04712 0.04718 ... 0.0558 0.05586 0.05592
    lat      (y, x) float64 136kB ...
    lon      (y, x) float64 136kB ...
  * y        (y) float64 856B 0.04225 0.0422 0.04214 ... 0.03643 0.03637 0.03632
  * time     (time) datetime64[ns] 16B 2020-01-01T01:05:07.617581056 2020-01-...
    x_image  (time) float32 8B ...
    y_image  (time) float32 8B ...
Data variables:
    CMI      (time, y, x) float64 272kB ...